# Ways to access satellite imagery with Copernicus Data Space Ecosystem

The Copernicus Data Space Ecosystem is
- Largest EO data offering in the world, with discovery and download capabilities
- A set of data processing tools to extract objective information and conduct public, private or commercial activities
- A thriving ecosystem to offer data, services and applications from public, commercial and scientific service providers
- A service to benefit institutional users, research, commercial sector as well as to every citizen of our planet

Let us understand how to download and visualize imagery with Jupyter lab in this notebook.

First, let us import all the important libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import getpass

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    bbox_to_dimensions,
    SentinelHubCatalog,
)

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

### Credentials

Credentials for Sentinel Hub services (`client_id` & `client_secret`) can be obtained by navigating to your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). In the User Settings you can create a new OAuth Client to generate these credentials. For more detailed instructions, visit the relevent [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Now that you have your `client_id` & `client_secret`, it is recommended to configure a new profile in your Sentinel Hub Python package. Instructions on how to configure your Sentinel Hub Python package can be found [here](https://sentinelhub-py.readthedocs.io/en/latest/configure.html). Using these instructions you can create a profile specific to using the package for accessing Copernicus Data Space Ecosystem data collections. This is useful as changes to the the config class are usually only temporary in your notebook and by saving the configuration to your profile you won't need to generate new credentials or overwrite/change the default profile each time you rerun or write a new Jupyter Notebook. 

If you are a first time user of the Sentinel Hub Python package for Copernicus Data Space Ecosystem, you should create a profile specific to the Copernicus Data Space Ecosystem. You can do this in the following cell:

In [ ]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
#config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
#config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")

However, if you have already configured a profile in Sentinel Hub Python for the Copernicus Data Space Ecosystem, then you can uncomment and run the below cell by entering the profile name as a string replacing `profile_name` (which is `cdse` in our case, feel free to change this when you save your profile).

In [ ]:
#config = SHConfig("cdse")

### Catalog API

[Catalog API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Catalog.html) is one of the services that allows custom data search options. It supports searches over all available data collections and provides the most information about each item in a collection. The implementation is based on STAC REST API specifications.

More information about the service is available in:

- [Documentation](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Catalog.html)
- [API definition](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/ApiReference.html#tag/catalog_core)

To check Data Availability in the Catalog, initialize the Sentinel hub Catalog API

In [ ]:
catalog = SentinelHubCatalog(config=config)

catalog.get_info()

Each data collection has its own catalog collection. The default collections are already listed in the info response. Information about all available collections, including user-defined BYOC and batch collections, can be obtained with the next example. We will try to display just the `id` of the collection.

In [ ]:
collections = catalog.get_collections()

collections = [collection['id'] for collection in collections]

collections

## Try it yourself!
Could you try and obtain information about `title`, `description`, and `providers` of these data collections?

Information about a single collection can also be obtained as follows.

In [ ]:
catalog.get_collection(DataCollection.SENTINEL1_EW)

### Search features in catalog collections

For any more advanced search we can use the `SentinelHubCatalog.search()` method. The method always requires a data collection, time interval and a bounding box or a geometry. Additionally, we can define various STAC-based query filters which are described in documentation and API definition.

In our case, we will filter all satellite tiles with more than 5% cloud coverage. To reduce the size of downloaded responses we will also filter fields that will be returned in the response.

In [ ]:
croatia_bbox = BBox((14.179862,44.953893,14.770494,45.359651), crs=CRS.WGS84).transform(3035)            # Convert to 3035 to get crs with meters as units
time_interval = "2023-04-01", "2023-09-01"

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
    bbox=croatia_bbox,
    time=time_interval,
    filter="eo:cloud_cover < 5",
    fields={"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

## Process API
The [Process API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html) is an essential element in the Copernicus Data Space Ecosystem. It allows the generation of personalized visual representations of satellite data and enables users to process and analyze data in the cloud. The Process API and Catalog API are two of the multiple Application Programming Interfaces (APIs) offered by the Copernicus Data Space Ecosystem. 

With [Catalog API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Catalog.html) we were able to get information about all satellites tiles for a given bounding box and time interval. Let's see how we can use this information with [Process API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html) to download data for all acquisitions. We can see that many timestamps differ only by a few seconds. That is because they are from tiles in the same orbit acquisition. Because of that, we want to join them together in a single timestamp.


### Example 1: True color (TIF) on a specific date

We build the request according to the [API Reference](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/ApiReference.html), using the `SentinelHubRequest` class. Each Process API request also needs an [evalscript](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Evalscript/V3.html).


The information that we specify in the `SentinelHubRequest` object is: 

 * an evalscript,
 * a list of input data collections with time interval,
 * a format of the response,
 * a bounding box and it's size (size or resolution).


#### Evalscript

An evalscript (or "custom script") is a piece of Javascript code which defines how the satellite data shall be processed by Sentinel Hub and what values the service shall return. It is a required part of any [Process](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html), [Batch processing](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Batch.html) or [OGC](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/OGC.html) request.

In the [Evalscript](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Evalscript.html) section you will find a technical documentation with detailed explanations of parameters and functions you can use in your evalscripts.

For an `evalscript` you need to specify two functions (described in detail below):

- `setup` - where you specify inputs and outputs.
- `evaluatePixel` - which calculates the output values for each pixel.


The evalscript in the example is used to select the appropriate bands. We return the RGB (B04, B03, B02) Sentinel-2 L2A bands.

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

In [ ]:
request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                "s2l2a", service_url=config.sh_base_url
            ),
            time_interval=time_interval,
            mosaicking_order=MosaickingOrder.MOST_RECENT,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=croatia_bbox,
    resolution=(50,50),
    config=config,
    data_folder="./data"
)

In [ ]:
plot_image(request_true_color.get_data(save_data=True)[0], factor=3.5 / 255, clip_range=(0, 1))

## Try it yourself!
Could you try and get a least cloudy image of this scene?

`Tip`: Try to change the mosaicking order to `LEAST_CC`.

## Try it yourself!
Could you try to change the evalscript to create a False Colour Composite of the least cloudy image obtained above using Near Infrared (`B08`), Red (`B04`) and Green (`B03`) bands? You will have to run the request again after changing the evalscript.

### Visualizing Remote Sensing Indices

There are many ways to analyse satellite imagery and one of those ways is to visaualise an index obtained by combining data from two or more bands. Let us try to visualize a Normalized Difference Water Index by combining the Blue (`B03`) and the NIR (`B08`) bands of Sentinel-2 L2A imagery. The following evalscript demonstrates visualizing water bodies in blue and displaying the land in true colour (RGB).

In [ ]:
evalscript_ndwi = """
//VERSION=3

function setup() {
  return {
    input: ["B02","B03","B04","B08", "dataMask"],
    output: [
      { bands: 3 }
    ]
  };
}

function evaluatePixel(samples) {
  let ndwi = (samples.B03-samples.B08)/(samples.B03+samples.B08);
  if (ndwi > 0){ 
    return[0,0,0.5];
    }
  else{
    return[2.5*samples.B04,2.5*samples.B03,2.5*samples.B02];
  }
}
"""

In [ ]:
request_index = SentinelHubRequest(
    evalscript=evalscript_ndwi,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                "s2l2a", service_url=config.sh_base_url
            ),
            time_interval=time_interval,
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=croatia_bbox,
    resolution=(50,50),
    config=config,
    data_folder="./data"
)

In [ ]:
plot_image(request_index.get_data(save_data=True)[0], factor=3.5 / 255, clip_range=(0, 1))

## Try it yourself!
Change the location to one with more vegetation (Example location: Mix of forest and agricultural fields in Austria `[16.106159,47.849928,16.238021,47.938325]`) and create a visualization showing the Normalized Difference Vegetation Index (NDVI) which combines the NIR (`B08`) and Red (`B04`) bands of Sentinel-2 imagery.

$$ NDVI = {B08 - B04 \over B08 + B04} $$